## 第二章：使用spaCy进行大规模数据分析

在本章中，我们会用一些新技术来从大量语料中抽取特定信息。 我们会学习如何利用spaCy的数据结构来结合统计与规则模型进行文本分析。

#### 数据结构 (1): Vocab, Lexemes和StringStore

欢迎回来！现在你已经有一些spaCy实例的实战经验了， 是时候学习一下spaCy背后到底是怎么工作的了。

在这门课中，我们要看下共享词汇表以及spaCy如何处理字符串。

#### 共享词汇表和字符串库 (1)

- `Vocab`: 存储那些多个文档共享的数据
- 为了节省内存使用，spaCy将所有字符串编码为**哈希值**。
- 字符串只在`StringStore`中通过`nlp.vocab.strings`存储一次。
- 字符串库：双向的**查询表**

```python
nlp.vocab.strings.add("咖啡")
coffee_hash = nlp.vocab.strings["咖啡"]
coffee_string = nlp.vocab.strings[coffee_hash]
```

- 哈希是不能逆求解的，所以我们要提供共享词汇表。

```python
# 如果该字符串从未出现过则会报错
string = nlp.vocab.strings[7962530705879205333]
```


### 安装下载spaCy和必要的库

```bash
# 确保已经安装了中文模型

pip install spacy
python -m spacy download zh_core_web_sm
```

In [8]:
import spacy

# 读取一个流程，创建nlp实例
nlp = spacy.load("zh_core_web_sm")

# nlp.vocab.strings.add("咖啡")
# coffee_hash = nlp.vocab.strings["咖啡"]
# coffee_string = nlp.vocab.strings[coffee_hash]

# # 如果该字符串从未出现过则会报错
# string = nlp.vocab.strings[7962530705879205333]

要拿到字符串的哈希值，我们要在`nlp.vocab.strings`中查找。

要拿到一个哈希值的字符串形式，我们可以查询哈希值。

一个`Doc`实例也可以暴露出它的词汇表和字符串。

In [9]:
# 在 nlp.vocab.strings 中查找字符串和哈希值

doc = nlp("我爱喝咖啡。")
print("hash value:", nlp.vocab.strings["咖啡"])
print("string value:", nlp.vocab.strings[7962530705879205333])

hash value: 7962530705879205333
string value: 咖啡


In [10]:
# doc 也会暴露出词汇表和字符串

doc = nlp("我爱喝咖啡。")
print("hash value:", doc.vocab.strings["咖啡"])

hash value: 7962530705879205333


#### Lexemes: 词汇表中的元素

- 一个`Lexeme`实例是词汇表中的一个元素

```python
doc = nlp("我爱喝咖啡。")
lexeme = nlp.vocab["咖啡"]

# 打印词汇的属性
print(lexeme.text, lexeme.orth, lexeme.is_alpha)
```

- 包含了一个词的

  和语境无关

  的信息

  - 词组的文本：`lexeme.text`和`lexeme.orth`（哈希值）
  - 词汇的属性如`lexeme.is_alpha`
  - **并不包含**和语境相关的词性标注、依存关系和实体标签

In [11]:
doc = nlp("我爱喝咖啡。")
lexeme = nlp.vocab["咖啡"]

# 打印词汇的属性
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

咖啡 7962530705879205333 True


#### Vocab, 哈希值和语素

!['I'、'love'和'coffee'三个词在Doc、Vocab和StringStore中的图解](https://course.spacy.io/vocab_stringstore_zh.png)

### 练习
 - 从字符串到哈希值
 - vocab（词汇表），哈希值和词素

#### 从字符串到哈希值

##### Part 1

- 在`nlp.vocab.strings`中查找字符串”猫”来得到哈希值。
- 查找这个哈希值来返回原先的字符串。

In [13]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp("我养了一只猫。")

# 查找词汇"猫"的哈希值
cat_hash = nlp.vocab.strings["猫"]
print(cat_hash)

# 查找cat_hash来得到字符串
cat_string = nlp.vocab.strings[cat_hash]
print(cat_string)

12262475268243743508
猫


##### Part 2

  - 在`nlp.vocab.strings`中查找字符串标签”PERSON”来得到哈希值。
  - 查找这个哈希值来返回原先的字符串。

In [14]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp("周杰伦是一个人物。")

# 查找标签是"人物"的字符串的哈希值
person_hash = nlp.vocab.strings["人物"]
print(person_hash)

# 查找person_hash来拿到字符串
person_string = nlp.vocab.strings[person_hash]
print(person_string)

16486493800568926464
人物


#### vocab（词汇表），哈希值和词素

> 分析这段代码抛出错误的原因：
> ```
> import spacy
> 
> # 创建一个英文和德文的nlp实例
> nlp = spacy.blank("en")
> nlp_de = spacy.blank("de")
> 
> # 获取字符串'Bowie'的ID
> bowie_id = nlp.vocab.strings["Bowie"]
> print(bowie_id)
> 
> # 在vocab中查找"Bowie"的ID
> print(nlp_de.vocab.strings[bowie_id])
> ```
> 
> 原因：The string "Bowie" 不在德语的vocab中，所以我们不能把哈希值转换为原始的字符串。 哈希值是不能逆求原始值的。为了解决这个问题， 我们要通过处理文本或者查找字符串把词组加入到新的vocab中， 或者使用同样的vocab把哈希值变回一个字符串。

### 数据结构(2)：Doc、Span和Token

我们已经学习了词汇表和字符串库，现在我们可以看下最重要的几个数据结构： 文档`Doc`、其视图词符`Token`以及跨度`Span`。

`Doc`是spaCy的核心数据结构之一。 当我们用`nlp`实例来处理文本时`Doc`就会被自动创建， 当然我们也可以手动初始化这个类。

创建`nlp`实例之后，我们就可以从`spacy.tokens`中导入`Doc`类。

这个例子中我们用了三个词来创建一个doc。空格存储在一个布尔值的列表中， 代表着对应位置的词后面是否有空格。每一个词符都有这个信息，包括最后一个词符！

`Doc`类有三个参数：共享的词汇表，词汇和空格。

In [15]:
# 创建一个nlp实例
import spacy
nlp = spacy.blank("en")

# 导入Doc类
from spacy.tokens import Doc

# 用来创建doc的词汇和空格
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# 手动创建一个doc
doc = Doc(nlp.vocab, words=words, spaces=spaces)

Span跨度实例(1)

一个`Span`是doc的一段包含了一个或更多的词符的截取。 `Span`类有最少三个参数：对应的doc以及span本身起始和终止的索引。 注意终止索引代表的词符是不包含在这个span里面的！

![Doc中的一个含有词符索引的Span实例图解](https://course.spacy.io/span_indices.png)

In [16]:
# 导入Doc和Span类
from spacy.tokens import Doc, Span

# 创建doc所需要的词汇和空格
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# 手动创建一个doc
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# 手动创建一个span
span = Span(doc, 0, 2)

# 创建一个带标签的span
span_with_label = Span(doc, 0, 2, label="GREETING")

# 把span加入到doc.ents中
doc.ents = [span_with_label]

最佳实践

- `Doc`和`Span`是非常强大的类，可以存储词语和句子的参考资料和关系。

  - **不到最后就不要把结果转换成字符串**
  - **尽可能使用词符属性**，比如用`token.i`来表示词符的索引

- 别忘了传入共享词汇表`vocab`

### 练习：创建一个Doc

创建 doc 和 span

In [17]:
import spacy

nlp = spacy.blank("en")

# 导入Doc类
from spacy.tokens import Doc

# 目标文本："spaCy is cool!"
words = ["spaCy", "is", "cool", "!"]
spaces = [True, True, False, False]

# 用words和spaces创建一个Doc
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

spaCy is cool!


In [18]:
import spacy

nlp = spacy.blank("en")

# 导入Doc类
from spacy.tokens import Doc

# 目标文本："Go, get started!"
words = ["Go", ",", "get", "started", "!"]
spaces = [False, True, True, False, False]

# 使用words和spaces创建一个Doc
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Go, get started!


In [20]:
import spacy

nlp = spacy.blank("en")

# 导入Doc类
from spacy.tokens import Doc

# 目标文本："Oh, really?!"
words = ["Oh", ",", "really", "?", "!"]
spaces = [False, True, False, False, False]

# 用words和spaces创建一个Doc
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Oh, really?!


### 练习：从头开始练习Docs（文档），spans（跨度）和entities（实体）

在这个练习中，我们要手动创建`Doc`和`Span`实例，然后更新命名实体。 实际中spaCy在后台也就是这么做的。 一个共享的`nlp`实例已经创建好了。

- 从`spacy.tokens`中导入`Doc`和`Span`。
- 用`Doc`类使用词组和空格直接创建一个`doc`实例。
- 用`doc`实例创建一个”David Bowie”的`Span`，赋予它`"PERSON"`的标签。
- 用一个实体的列表，也就是”David Bowie” `span`，来覆盖`doc.ents`。

In [ ]:
import spacy

nlp = spacy.blank("zh")

# 导入Doc和Span类
from spacy.tokens import Doc, Span

words = ["我", "喜欢", "周", "杰伦"]
spaces = [False, False, False, False]

# 用words和spaces创建一个doc
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

# 为doc中的"周杰伦"创建一个span，并赋予其"PERSON"的标签
span = Span(doc, 2, 4, label="PERSON")
print(span.text, span.label_)

# 把这个span加入到doc的实体中
doc.ents = [span]  # 也可以用doc.ents = list(doc.ents) + [span]

# 打印所有实体的文本和标签
print([(ent.text, ent.label_) for ent in doc.ents])

上面内容很重要，在之后我们学习编码信息提取流程的时候，我们就会发现手动创建spaCy的实例并改变其中的实体会非常方便有用。

数据结构最佳实践

```py
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp("北京是一座美丽的城市")

# 获取所有的词符和词性标注
token_texts = [token.text for token in doc]
pos_tags = [token.pos_ for token in doc]

for index, pos in enumerate(pos_tags):
    # 检查当前词符是否是专有名词
    if pos == "PROPN":
        # 检查下一个词符是否是动词
        if pos_tags[index + 1] == "VERB":
            result = token_texts[index]
            print("Found proper noun before a verb:", result)
```


In [21]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp("北京是一座美丽的城市")

# 获取所有的词符和词性标注
token_texts = [token.text for token in doc]
pos_tags = [token.pos_ for token in doc]

for index, pos in enumerate(pos_tags):
    # 检查当前词符是否是专有名词
    if pos == "PROPN":
        # 检查下一个词符是否是动词
        if pos_tags[index + 1] == "VERB":
            result = token_texts[index]
            print("Found proper noun before a verb:", result)

Found proper noun before a verb: 北京


注意！！ 上面内容虽然是正确的，但是最好不要在最终结果输出之前就将结果转换成字符串形式。应该要避免把词符变成字符串，因为这样的话我们就不能**够读取其属性和关系了**。

第二部分

- 用原生的词符属性而不是`token_texts`和`pos_tags`的列表来重写代码。
- 在`doc`中遍历每一个`token`并检查其`token.pos_`属性。
- 用`doc[token.i + 1]`来检查下一个词符及其`.pos_`属性
- 如果找到一个处于动词前的专有名词，我们就打印其`token.text`。

In [23]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp("北京是一座美丽的城市")

# 遍历所有字符
for token in doc:
    # 检查当前字符是否是专有名词
    if token.pos_ == "PROPN":
        # 检查下一个字符是否是动词
        if doc[token.i + 1].pos_ == "VERB":
            print("找到了动词前面的一个专有名词：", token.text)

找到了动词前面的一个专有名词： 北京


虽然上面代码例子表现不错，但还是有很多改进空间。
如果doc是由一个专有名词结尾的，`doc[token.i + 1]` 就会报错。为了保证我们的代码能适用于更多场景，我们需要首先检查下是否 `token.i + 1 < len(doc)`

### 词向量和语义相似度计算（重要）

本节课中我们要学习如何用spaCy来判断 文档document、跨度span或者词符token之间有多相似。

我们还会学到如何在实际的自然语言处理应用中用到词向量。

非常重要的一点：**要计算相似度，我们必须需要一个比较大的含有词向量的spaCy流程。**

- `spaCy`可以对比两个实例来判断它们之间的相似度

- `Doc.similarity()`、`Span.similarity()`和`Token.similarity()`

- 使用另一个实例作为参数返回一个相似度分数(在`0`和`1`之间)

- 注意：

  我们需要一个含有词向量的流程，比如：

  - ✅ `en_core_web_md` (中等)
  - ✅ `en_core_web_lg` (大)
  - 🚫 **而不是** `en_core_web_sm` (小)

In [39]:
# 相似度举例

# 读取一个有词向量的较大流程
import spacy

nlp = spacy.load("zh_core_web_md")

# 比较两个文档
doc1 = nlp("怎么解锁宠物之家")
doc2 = nlp("怎么开启宠物的家？")
print(doc1.similarity(doc2))

0.793683571249384


In [40]:
# 比较两个词符
doc = nlp("西红柿和番茄是一样的")

print([text for text in doc])

token1 = doc[2]
token2 = doc[0]
print(token1.similarity(token2))

[西红柿, 和, 番茄, 是, 一样, 的]
0.41378068923950195


In [49]:
# 对比一篇文章和一个词符
doc = nlp("怎么解锁宠物之家")
token = nlp("家")[0]

print(doc.similarity(token))

0.5687867812442264


In [47]:
# 对比一个跨度span和一篇文档
span = nlp("怎么解锁宠物的家")[2:5]
print([text for text in span])

doc = nlp("如何开启宠物之家")

print(span.similarity(doc))

[宠物, 的, 家]
0.7687521259814877


### spaCy是如何判断相似度的？

- 相似度是通过**词向量**计算的
- 词向量是一个词汇的多维度的语义表示
- 词向量是用诸如[Word2Vec](https://en.wikipedia.org/wiki/Word2vec) 这样的算法在大规模语料上面生成的
- 词向量可以是spaCy流程的一部分
- 默认我们使用余弦相似度，但也有其它计算相似度的方法
- `Doc`和`Span`的向量默认是由其词符向量的平均值计算得出的
- 短语的向量表示要优于长篇文档，因为后者含有很多不相关的词

spaCy在后台到底是怎么做到相似度计算的？

相似度是通过词向量计算的，词向量是一个词汇的多维度的语义表示。

可能你听说过Word2Vec，这就是常常被用来从原始语料中训练出词向量的其中一种算法。

词向量可以是spaCy流程的一部分。

默认spaCy会返回两个向量的余弦相似度，但有需要时我们也可以替换为其它计算相似度的方法。

一个包含了多个词符的实例，比如Doc和Span， 默认的向量值的计算方法是其中所有词符向量的平均值。

这也是为什么通常**短语**的向量更有价值因为其中不相关的词会比较少。

#### spaCy中的词向量

这个例子能让我们大致了解这些向量长什么样。

首先我们再次读取中等大小的流程，这个流程含有词向量。

然后我们处理一段文本，用.vector属性来查找一个词符的向量。

结果是"banana"这个词的一个300维的向量。

In [53]:
# 导入一个含有词向量的较大的流程
nlp = spacy.load("zh_core_web_lg")

doc = nlp("怎么快速升级")
print([text for text in doc])

# 通过token.vector属性获取向量
print(doc[1:3].vector)

[怎么, 快速, 升级]
[-9.39499855e-01 -5.93200028e-01  2.31710005e+00  2.77328491e+00
 -1.34800017e-01 -8.59369993e-01 -4.55399990e+00  2.89321494e+00
  7.53765011e+00  2.06819987e+00  1.26613998e+00 -4.04060006e-01
 -2.85529995e+00  5.92665017e-01  1.09713006e+00 -1.07256508e+00
  2.08375001e+00 -3.00470018e+00 -2.02804494e+00 -1.18826497e+00
  1.76103950e+00  2.19831514e+00  6.31200016e-01 -9.24480081e-01
 -1.89569998e+00  1.64084995e+00 -1.80553007e+00 -6.46590054e-01
 -4.35134983e+00 -4.70749974e-01 -1.77869999e+00  1.98154497e+00
  1.51329494e+00 -7.25255013e-01 -2.05837989e+00 -6.46412015e-01
 -2.23334998e-01  3.01529002e+00 -4.66424990e+00 -6.59861982e-01
 -2.69689989e+00  1.92429996e+00 -2.08240008e+00 -3.80239987e+00
  5.43229997e-01  3.21155000e+00 -1.86499953e-02 -1.31601501e+00
 -3.75600010e-01  2.30875015e+00 -1.38937998e+00 -2.38456011e+00
 -2.46749997e+00  1.16578007e+00 -1.04484999e+00 -1.93719995e+00
 -4.76029968e+00 -1.22871494e+00 -1.85000896e-03 -1.53197002e+00
  2.26959991

基于应用场景的相似度

- 对很多应用都很有用，比如推荐系统、查重系统等
- “相似度”并没有一个客观的定义方法
- 主要还是决定于实际场景和所支持的应用是要做什么

```py
doc1 = nlp("I like cats")
doc2 = nlp("I hate cats")

print(doc1.similarity(doc2))
>>>> 0.9501447503553421
```

我们看一个例子：spaCy默认的词向量会对"I like cats"和"I hate cats"这两句哈给出非常高的相似度分数。

这是有道理的，因为两个文本都在讲关于猫的看法。

但在另一个应用场景里，你可能希望这两句话是 *非常不同* 的，因为它们的看法是完全相反的。

### 练习：检查词向量

In [54]:
import spacy

# 读取zh_core_web_md流程
nlp = spacy.load("zh_core_web_md")

# 处理文本
doc = nlp("两只老虎跑得快")

for token in doc:
    print(token.text)

# 获取词符"老虎"的向量
laohu_vector = doc[2].vector
print(laohu_vector)

两
只
老虎
跑
得
快
[ 5.7924e-01 -2.6305e-01 -1.4191e-01 -5.0995e+00  3.8716e+00  3.5153e+00
 -6.9870e-01  2.6803e+00 -9.4611e-01  5.0214e+00  1.6222e+00  1.5286e+00
  8.7571e-01  2.6110e+00  2.3262e-01  2.5249e+00 -1.6588e+00  1.8337e+00
 -2.5249e+00  3.8427e+00  2.6680e+00 -8.2123e-01  1.6126e+00 -3.7706e+00
  3.8015e+00  7.8155e-02  1.5115e+00  2.8359e-01  3.1309e+00  1.5774e+00
 -5.5651e-01 -1.7239e+00 -3.7953e+00 -6.6034e-01 -9.2233e-01  4.9122e-01
 -1.4692e+00 -2.8478e+00 -4.9413e+00 -2.4462e+00  1.6943e+00  2.3306e+00
  8.0750e-01  2.1319e+00  3.7470e+00  2.7392e+00  3.2851e+00  3.2695e+00
  2.8855e+00 -1.7605e+00 -5.0122e-01 -3.4106e-01 -3.4631e+00  7.9958e-01
 -1.7841e+00  1.5095e-01 -2.5067e+00  4.1346e-01 -3.0813e-01  6.0523e-02
 -1.8828e+00  8.6008e-01 -2.3530e+00  1.5310e+00  5.9555e-01 -1.6424e+00
 -1.4507e+00  1.9775e+00 -2.0973e+00 -4.8088e-01 -6.6589e-01  1.2523e+00
  1.5388e+00 -3.8856e+00 -1.0470e+00 -6.1471e-01 -1.2202e+00  3.0140e+00
 -2.5228e+00 -2.5406e+00 -3.6240e-01  

### 练习：对比相似度

在这个练习中，使用spaCy的`similarity`方法来比较`Doc`、`Token`和`Span`实例，得到相似度分数。

第一部分

- 使用`doc.similarity`方法来比较`doc1`和`doc2`的相似度并打印结果。


In [55]:
import spacy

nlp = spacy.load("zh_core_web_md")

doc1 = nlp("这是一个温暖的夏日")
doc2 = nlp("外面阳光明媚")

# 获取doc1和doc2的相似度
similarity = doc1.similarity(doc2)
print(similarity)

0.5488376705728557


第二部分

- 使用`token.similarity`方法来比较`token1`和`token2`的相似度并打印结果。


In [56]:
import spacy

nlp = spacy.load("zh_core_web_md")

doc = nlp("电影和音乐")

for i, token in enumerate(doc):
    print(i, token.text)

token1, token2 = doc[0], doc[2]

# 获取词符"TV"和"books"的相似度
similarity = token1.similarity(token2)
print(similarity)

0 电影
1 和
2 音乐
0.32749298214912415


第三部分

- 为”不错的餐厅”/“很好的酒吧”创建跨度(span)。
- 使用`span.similarity`来比较它们并打印结果。

In [57]:
import spacy

nlp = spacy.load("zh_core_web_md")

doc = nlp("这是一家不错的餐厅。之后我们又去了一家很好的酒吧。")

for i, token in enumerate(doc):
    print(i, token.text)

# 给"great restaurant"和"really nice bar"分别创建span
span1 = doc[2:5]
span2 = doc[12:15]

# 获取两个span的相似度
similarity = span1.similarity(span2)
print(similarity)

0 这是
1 一家
2 不错
3 的
4 餐厅
5 。
6 之后
7 我们
8 又
9 去
10 了
11 一家
12 很
13 好的
14 酒吧
15 。
0.6806249618530273


注意：实际开发一些自然语言处理的应用并用到语义相似度可能需要在自己的数据上 *先训练词向量 再去改进一下相似度的算法。*

### 流程和规则的结合

统计模型vs规则

|               | **统计模型**                           | **规则系统**                      |
| ------------- | -------------------------------------- | --------------------------------- |
| **应用场景**  | 需要根据例子来 *泛化* 的应用           | ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⠀⠀⠀⠀ |
| **真实范例**  | 产品名、人名、主语宾语关系             |                                   |
| **spaCy功能** | 实体识别器、依存句法识别器、词性标注器 |     

统计预测vs规则

|               | **统计模型**                           | **规则系统**                         |
| ------------- | -------------------------------------- | ------------------------------------ |
| **使用场景**  | 需要根据例子来 *泛化* 的应用           | 有限个例子组成的字典                 |
| **真实范例**  | 产品名、人名、主宾关系                 | 世界上的国家、城市、药品名、狗的种类 |
| **spaCy功能** | 实体识别器、依存句法识别器、词性标注器 | 分词器, `Matcher`, `PhraseMatcher`   |

回顾：基于规则的匹配

```python
# 用共享词汇表初始化
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# 模板是一个代表词符的字典组成的列表
pattern = [{"LEMMA": "love", "POS": "VERB"}, {"LOWER": "cats"}]
matcher.add("LOVE_CATS", [pattern])

# 运算符可以定义一个词符应该被匹配多少次
pattern = [{"TEXT": "very", "OP": "+"}, {"TEXT": "happy"}]
matcher.add("VERY_HAPPY", [pattern])

# 在doc上面调用matcher来返回一个(match_id, start, end)元组的列表
doc = nlp("I love cats and I'm very very happy")
matches = matcher(doc)
```

In [60]:
# 用共享词汇表初始化
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# 模板是一个代表词符的字典组成的列表
pattern = [{"LOWER": "喜欢"}]
matcher.add("LOVE_CATS", [pattern])

# 运算符可以定义一个词符应该被匹配多少次
pattern = [{"TEXT": "史瓦西", "OP": "+"}, {"TEXT": "史瓦西"}]
matcher.add("VERY_HAPPY", [pattern])

# 在doc上面调用matcher来返回一个(match_id, start, end)元组的列表
doc = nlp("我喜欢史瓦西！")
matches = matcher(doc)

统计预测的加成

In [62]:
matcher = Matcher(nlp.vocab)
matcher.add("NPC", [[{"LOWER": "史瓦西"}]])
doc = nlp("我喜欢史瓦西的发型！")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print("Matched span:", span.text)
    # 获取span的根词符和根头词符
    print("Root token:", span.root.text)
    print("Root head token:", span.root.head.text)
    # 获取前一个词符及其词性标注的POS标签
    print("Previous token:", doc[start - 1].text, doc[start - 1].pos_)

Matched span: 史瓦西
Root token: 史瓦西
Root head token: 发型
Previous token: 喜欢 VERB


高效短语匹配

- `PhraseMatcher`和普通正则表达式或者关键词搜索类似，但是可以直接读取词符！
- 将`Doc`实例作为模板
- 比`Matcher`更快更高效
- 适用于大规模词表的匹配

In [63]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("史瓦西")
matcher.add("NPC", [pattern])
doc = nlp("我喜欢史瓦西的发型！好酷一女熊")

# 遍历匹配结果
for match_id, start, end in matcher(doc):
    # 获取匹配到的span
    span = doc[start:end]
    print("Matched span:", span.text)

Matched span: 史瓦西


短语匹配器phrase matcher可以从`spacy.matcher`中导入，和普通的matcher是一样的API。

我们传进一个`Doc`实例而不是字典列表作为模板。

然后我们就可以遍历文本中的匹配结果，这些结果中有匹配的ID和匹配的起始和终止索引。 同时也可以让我们可以创建一个匹配到的词符"史瓦西"的`Span`实例使我们可以做情景中的分析。

#### 练习：模板调试1

为什么这个模板不能匹配到`doc`中的词符”Silicon Valley”？

```python
pattern = [{"LOWER": "silicon"}, {"TEXT": " "}, {"LOWER": "valley"}]
```

```python
doc = nlp("Can Silicon Valley workers rein in big tech from within?")
```
分词器不能为单空格创建词符，所以文本中的空格" "并没有变为词符。

#### 练习：模板调试2

这个练习中的两个模板都出错了，匹配不到我们想要的结果。 你能改正它们吗？要是你卡住了，可以尝试把`doc`中的词符打印出来， 看看这些文本应该怎样被分割，然后调整你的模板保证每个字典表示一个词符。

- 编辑`pattern1`使其可以正确匹配到所有的形容词后面跟着`"笔记本"`。
- 编辑`pattern2`使其可以正确匹配到`"锐龙"`加上后面的数字 (LIKE*NUM) 和符号 (IS*ASCII) 。

In [64]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("zh_core_web_sm")

doc = nlp("荣耀将于7月16日发布新一代 MagicBook 锐龙笔记本，显然会配备7nm工艺、Zen2 架构的"
          "全新锐龙4000系列，但具体采用低功耗的锐龙4000U 系列，还是高性能的锐龙4000H 系列，"
          "目前还没有官方消息。今天，推特曝料大神公布了全新 MagicBook Pro 锐龙本的配置情况。"
         )

# 创建匹配模板
pattern1 = [{"POS": "ADJ"},{"TEXT": "笔记本"}]
pattern2 = [{"TEXT": "锐龙"}, {"LIKE_NUM": True}, {"IS_ASCII": True}]

# 初始化matcher并加入模板
matcher = Matcher(nlp.vocab)
matcher.add("PATTERN1", [pattern1])
matcher.add("PATTERN2", [pattern2])

# 遍历匹配结果
for match_id, start, end in matcher(doc):
    # 打印匹配到的字符串名字及匹配到的span的文本
    print(doc.vocab.strings[match_id], doc[start:end].text)

PATTERN2 锐龙4000U
PATTERN2 锐龙4000H


#### 练习：高效率的短语匹配

有时候相比起写一些描述单个词符的模板，直接精确匹配字符串可能更高效。 在面对有限个种类的东西时尤其如此，比如世界上的所有国家。 

我们已经有一个国家的列表，所以我们用它作为我们信息提取代码的基础。 变量`COUNTRIES`中存取了这些字符串名字的列表。

- 导入`PhraseMatcher`并用含有共享`vocab`的变量`matcher`来初始化。
- 加入短语模板并在’doc’上面调用matcher

In [68]:
import json
import spacy

with open("countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())

nlp = spacy.blank("zh")
doc = nlp("智利可能会从斯洛伐克进口货物")

# 导入PhraseMatcher并实例化
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

# 创建Doc实例的模板然后加入matcher中
# 下面的代码比这样的表达方式更快： [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", patterns)

# 在测试文档中调用matcher并打印结果
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

[智利, 斯洛伐克]


#### 练习：提取国家和关系

在上一个练习中，我们写了一段代码用spaCy的`PhraseMatcher`来寻找文本中的国家名。 我们现在用这个国家匹配器来匹配一段更长的文本，分析句法， 并用匹配到的国家名更新文档中的实体。

- 对匹配结果进行遍历， 创建一个标签为`"GPE"`(geopolitical entity，地理政治实体)的`Span`。
- 覆盖`doc.ents`中的实体，加入匹配到的跨度span。
- 获取匹配到的跨度span中的根词符的头。
- 打印出词符头和跨度span的文本。

In [69]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import json

with open("countries.json", encoding="utf8") as f:
    COUNTRIES = json.loads(f.read())
with open("country_text.txt", encoding="utf8") as f:
    TEXT = f.read()

nlp = spacy.load("zh_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", patterns)

# 创建一个doc并重置其已有的实体
doc = nlp(TEXT)
doc.ents = []

# 遍历所有的匹配结果
for match_id, start, end in matcher(doc):
    # 创建一个标签为"GPE"的span
    span = Span(doc, start, end, label="GPE")

    # 覆盖doc.ents并添加这个span
    doc.ents = list(doc.ents) + [span]

    # 获取这个span的根头词符
    span_root_head = span.root.head
    # 打印这个span的根头词符的文本及span的文本
    print(span_root_head.text, "-->", span.text)

# 打印文档中的所有实体
print([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "GPE"])

内战 --> 萨尔瓦多
任务 --> 纳米比亚
统治 --> 南非
红色 --> 柬埔寨
授权 --> 美国
入侵 --> 伊拉克
入侵 --> 科威特
莫桑比克 --> 索马里
莫桑比克 --> 海地
南斯拉夫 --> 莫桑比克
包括 --> 南斯拉夫
损失 --> 美国
行动 --> 索马里
援助团 --> 卢旺达
大屠杀 --> 卢旺达
欧洲 --> 美国
总统 --> 美国
新加坡 --> 英国
紧随 --> 新加坡
撤资 --> 美国
任务 --> 塞拉利昂
陆战队 --> 英国
入侵 --> 阿富汗
入侵 --> 美国
入侵 --> 伊拉克
冲突 --> 苏丹
共和国 --> 刚果
内战 --> 叙利亚
末期 --> 斯里兰卡
地震 --> 海地
[('萨尔瓦多', 'GPE'), ('纳米比亚', 'GPE'), ('南非', 'GPE'), ('柬埔寨', 'GPE'), ('美国', 'GPE'), ('伊拉克', 'GPE'), ('科威特', 'GPE'), ('索马里', 'GPE'), ('海地', 'GPE'), ('莫桑比克', 'GPE'), ('南斯拉夫', 'GPE'), ('美国', 'GPE'), ('索马里', 'GPE'), ('卢旺达', 'GPE'), ('卢旺达', 'GPE'), ('美国', 'GPE'), ('美国', 'GPE'), ('英国', 'GPE'), ('新加坡', 'GPE'), ('美国', 'GPE'), ('塞拉利昂', 'GPE'), ('英国', 'GPE'), ('阿富汗', 'GPE'), ('美国', 'GPE'), ('伊拉克', 'GPE'), ('苏丹', 'GPE'), ('刚果', 'GPE'), ('叙利亚', 'GPE'), ('斯里兰卡', 'GPE'), ('海地', 'GPE')]
